# PTST Idealization using axi-symmetric 2D interface layer

## Purpose of the notebook

Verify the axi-symmetric model in an elastic state to ensure that the confinement is
correctly reproduced. This notebook serves as a basis for 3D degradation within
a ligament/bond zone in axi-symmetric models. Examples are
 - PTST
 - prestressing
 - confined bond configuration
 - effect of splitting cracks
 - restrained splitting cracks in fatigue tests

## Implementation remarks

- The notebook now uses `BCSliceI` - to ensure the same order of nodes along an 
interfaces
- Compression is applied using individual `BCDof` instances without integration 
  over the boundary. This should be changed to BCSliceE - which can use the 
  finite element shape functions to integrate over the element domain

In [28]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSliceI, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic

## FE Axisymmetric discretization

In [40]:
n_x_e = 20
n_inner_y_e = 10
n_outer_y_e = 10
L_x = 100.0 # [mm]
d_x = L_x / n_x_e
R_in = 25 # [mm]
dR = 10 # [mm]
R_out = 50 # [mm]
len_notch = 10
n_notch = 2
# geo_transform
# gmesh
xd_inner = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, 0),
                               coord_max=(L_x, R_in),
                               shape=(n_x_e, n_inner_y_e),
                               fets=FETS2D4Q())
xd_middle = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0+n_notch*d_x, R_in),
                               coord_max=(L_x-n_notch*d_x, R_in+dR),
                               shape=(n_x_e-2*n_notch, 1),
                               fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, R_in+dR),
                               coord_max=(L_x, R_out),
                               shape=(n_x_e, n_outer_y_e),
                               fets=FETS2D4Q())
m_elastic = MATS3DElastic(E=37000, nu=0.18)
m_inelastic = MATS3DElastic(E=37000, nu=0.18)
m = TStepBC(
    domains=[(xd_inner, m_elastic),
             (xd_outer, m_elastic),
             (xd_middle, m_inelastic),
             ]
)

## Boundary and transition conditions

### Kinematic links between the domains

In [41]:
link_inner_middle = BCSliceI(var='u', dims=[0, 1],
                             slice=xd_inner.mesh.I[n_notch:-n_notch, -1], 
                             link_slice=xd_middle.mesh.I[:, 0],
                             link_coeffs=[1.0, 1.0],
                             link_dims=[0, 1],
                             value=0.0
                            )
link_middle_outer = BCSliceI(slice=xd_outer.mesh.I[n_notch:-n_notch, 0], 
                             var='u', dims=[0, 1],
                             link_slice=xd_middle.mesh.I[:, -1],
                             link_coeffs=[1, 1],
                             link_dims=[0, 1],
                             value=0,
                            )

### Compression/Traction fc

In [42]:
inner_fixed_1 = BCSliceI(slice=xd_inner.mesh.I[:, 0], var='u', dims=[1], value=0)
outer_fixed_0 = BCSliceI(slice=xd_outer.mesh.I[0, :], var='u', dims=[0], value=0)

outer_compression_slice = BCSliceI(slice=xd_outer.mesh.I[:, -1],
                                   var='u', dims=[1], value=0.01)

compression_dofs = outer_compression_slice.dofs
compression = -10000*L_x / len(compression_dofs)
outer_compression_force_first = [BCDof(var='f', dof=dof, value=compression) 
                                 for dof in compression_dofs ]
outer_compression_force_first[0].value *=0.5
outer_compression_force_first[-1].value *=0.5
bc1 = [inner_fixed_1, 
       outer_fixed_0,
       link_inner_middle,
       link_middle_outer
      ] + outer_compression_force_first

### Sliding dc

In [43]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=6)
#
#
#bc1 =   [lower_fixed_0, upper_fixed_1, lower_slide]

### Sliding fc

In [44]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#Force = 10 / len(lower_slide.dofs)
#lower_slide_force = [BCDof(var='f', dof=dof, value = Force) 
#             for dof in lower_slide.dofs]
#
#bc1 =   [lower_fixed_0, upper_fixed_1] + lower_slide_force

In [45]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains

In [46]:
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



## Postprocesing

### Access the last displacement step

In [47]:
U_last = m.hist.U_t[-1]

In [48]:
U_last[xd_middle.o_Ia], U_last[xd_inner.o_Ia]

(array([[ 0.01061926, -0.01684768],
        [ 0.00837729, -0.03542182],
        [ 0.01176449, -0.02017661],
        [ 0.01011368, -0.02954748],
        [ 0.01249317, -0.01965715],
        [ 0.01108955, -0.02720484],
        [ 0.01330576, -0.01874812],
        [ 0.0120689 , -0.02553631],
        [ 0.0141892 , -0.01785784],
        [ 0.01313311, -0.02436855],
        [ 0.015128  , -0.0171437 ],
        [ 0.01430671, -0.02357273],
        [ 0.01610819, -0.01663247],
        [ 0.01557559, -0.02305466],
        [ 0.01711172, -0.01631337],
        [ 0.01690946, -0.02274   ],
        [ 0.01811522, -0.01617161],
        [ 0.0182722 , -0.02258426],
        [ 0.01909116, -0.01620656],
        [ 0.01962481, -0.02257501],
        [ 0.02000991, -0.01643842],
        [ 0.02092302, -0.02273421],
        [ 0.02084311, -0.01690588],
        [ 0.02211354, -0.02312254],
        [ 0.02156926, -0.01765012],
        [ 0.02313306, -0.02384562],
        [ 0.02218394, -0.01867627],
        [ 0.02391303, -0.025

In [49]:
xd_middle.map_U_to_field(U_last)

array([[[[ 2.54029976e-04, -2.44257087e-04,  0.00000000e+00],
         [-2.44257087e-04, -1.66292613e-03,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -7.76939114e-04]],

        [[ 2.54029976e-04, -2.27191828e-04,  0.00000000e+00],
         [-2.27191828e-04, -1.13157517e-03,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -8.06411083e-04]],

        [[ 3.22291014e-04,  3.04159130e-04,  0.00000000e+00],
         [ 3.04159130e-04, -1.13157517e-03,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -8.63495947e-04]],

        [[ 3.22291014e-04,  2.87093871e-04,  0.00000000e+00],
         [ 2.87093871e-04, -1.66292613e-03,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -9.32480424e-04]]],


       [[[ 1.56183815e-04,  1.05458891e-05,  0.00000000e+00],
         [ 1.05458891e-05, -8.98558954e-04,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -8.10146841e-04]],

        [[ 1.56183815e-04,  1.76818058e-05,  0.00000000e+0

In [ ]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()